In [1]:
import cv2
import numpy as np

# 예제 7.8 cv2.floodFill() 영역 채우기
### <span style='background-color:#fff5b1'>cv2.floodFill(image, mask, seedPoint, newVal)</span>
image에서 seedPoint 점에서 시작하여 물체 내부를 특정 값으로 채움 \
채워진 영역의 바운딩 사각형 rect 반환 \
**mask** 8비트 단일 채널, 크기는 image보다 가로 세로 각각 2씩 큼

In [2]:
src = np.full((512,512,3), (255,255,255), dtype=np.uint8)
cv2.rectangle(src, (50,50), (200,200), (0,0,255), 2)
cv2.circle(src, (300,300), 100, (0,0,255), 2)

dst = src.copy()
cv2.floodFill(dst, mask = None, seedPoint = (100,100), newVal = (255,0,0))

retval, dst2, mask, rect = cv2.floodFill(dst, mask = None, seedPoint = (300,300), newVal = (0,255,0))
print(rect)
x, y, width, height = rect
cv2.rectangle(dst2, (x,y), (x+width, y+height), (255,0,0), 2)

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

(202, 202, 197, 197)


# 예제 7.9 cv2.distanceTransform() 거리 계산
### <span style='background-color:#fff5b1'>cv2.distanceTransform(image, distanceType, maskSize)</span>
영상 영역의 내부의 0이 아닌 화소에서 가장 가까운 0인 화소까지의 거리 계산하여 실수형 배열 dst에 반환 \
**src** 1-채널 8비트 이진 영상 \
**distanceType** 거리 계산 종류(DIST_L1, DIST_L2, DIST_C) \
**maskSize** 마스크 크기(3, 5, DIST_MASK_PRECISE)

In [3]:
src = np.zeros(shape = (512,512), dtype = np.uint8)
cv2.rectangle(src, (50,200), (450,300), (255,255,255), -1)

dist = cv2.distanceTransform(src, distanceType = cv2.DIST_L1, maskSize = 3)
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(dist)
print(minVal, maxVal, minLoc, maxLoc)
dst = cv2.normalize(dist, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
ret, dst2 = cv2.threshold(dist, maxVal-1, 255, cv2.THRESH_BINARY)

gx = cv2.Sobel(dist, cv2.CV_32F, 1, 0, ksize = 3)
gy = cv2.Sobel(dist, cv2.CV_32F, 0, 1, ksize = 3)
mag = cv2.magnitude(gx, gy)
minVal, maxVal, minLoc, maxLoc = cv2.minMaxLoc(mag)
print(minVal, maxVal, minLoc, maxLoc)
ret, dst3 = cv2.threshold(mag, maxVal-2, 255, cv2.THRESH_BINARY_INV)

cv2.imshow('src', src)
cv2.imshow('dst', dst)
cv2.imshow('dst2', dst2)
cv2.imshow('dst3', dst3)
cv2.waitKey()
cv2.destroyAllWindows()

0.0 51.0 (0, 0) (100, 250)
0.0 8.0 (0, 0) (52, 200)


# 예제 7.10 cv2.watershed() 영상 분할
### <span style='background-color:#fff5b1'>cv2.watershed(image, markers)</span>
마커 기반 영상 분할 \
3-채널 8비트 컬러 image에 사용자가 대략적으로 markers에 부분영역을 설정하면 영상을 분할하여 markers 배열 반환 \
**markers** 1-채널 32비트 정수 \
초기에 markers에 주어진 영역의 값을 seed로 하여 나머지 영역을 분할함

In [6]:
src = cv2.imread('../sample/hand.jpg')
mask = np.zeros(shape = src.shape[:2], dtype = np.uint8)
markers = np.zeros(shape = src.shape[:2], dtype = np.int32)
dst = src.copy()
cv2.imshow('dst', dst)

def onMouse(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            cv2.circle(param[0], (x,y), 10, (255,255,255), -1)
            cv2.circle(param[1], (x,y), 10, (255,255,255), -1)
    cv2.imshow('dst', param[1])
    
mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
while True:
    cv2.setMouseCallback('dst', onMouse, [mask, dst])
    key = cv2.waitKey(300)
    if key == 0x1b:
        break
    elif key == ord('r'):
        mask[:,:] = 0
        dst = src.copy()
        cv2.imshow('dst', dst)
    elif key == ord(' '):
        contours, hierarchy = cv2.findContours(mask, mode, method)
        print(len(contours))
        markers[:,:] = 0
        for i, cnt in enumerate(contours):
            cv2.drawContours(markers, [cnt], 0, i + 1, -1)
        cv2.watershed(src, markers)
        dst = src.copy()
        dst[markers == -1] = [0,0,255]
        for i in range(len(contours)):
            r = np.random.randint(256)
            g = np.random.randint(256)
            b = np.random.randint(256)
            dst[markers == i+1] = [b,g,r]

            dst = cv2.addWeighted(src, 0.4, dst, 0.6, 0)
            cv2.imshow('dst', dst)

cv2.destroyAllWindows()

1
2
3


# 예제 7.11 cv2.distanceTransform(), cv2.watershed() 영상 분할